In [ ]:
# Step 1: Import necessary libraries
# --------------------------------------
# Import required libraries for document retrieval, reranking, and logging setup.
from importlib import reload
from sentence_transformers import CrossEncoder
import logging

import financerag.tasks.Base_Task; reload(financerag.tasks.Base_Task)
import financerag.tasks.ConvFinQATask; reload(financerag.tasks.ConvFinQATask)
import financerag.tasks.FinanceBenchTask; reload(financerag.tasks.FinanceBenchTask)
import financerag.tasks.FinDERTask; reload(financerag.tasks.FinDERTask)
import financerag.tasks.FinQATask; reload(financerag.tasks.FinQATask)
import financerag.tasks.FinQABenchTask; reload(financerag.tasks.FinQABenchTask)
import financerag.tasks.MultiHierttTask; reload(financerag.tasks.MultiHierttTask)
import financerag.tasks.TATQATask; reload(financerag.tasks.TATQATask)
reload(financerag.tasks)

from financerag.rerank import CrossEncoderReranker
from financerag.retrieval import DenseRetrieval, SentenceTransformerEncoder
from financerag.tasks import FinDER, ConvFinQA

# Setup basic logging configuration to show info level messages.
logging.basicConfig(level=logging.INFO)



In [ ]:
# Corpus Test

'''import json
import os

query_path = os.path.join(os.getcwd(), 'data', 'convfinqa', 'corpus.jsonl')

with open(query_path, 'r', encoding='utf-8') as f:
    queries = [json.loads(line) for line in f]

queries[0]['text']'''

'containerboard , kraft papers and saturating kraft .\nkapstone also owns victory packaging , a packaging solutions distribution company with facilities in the u.s. , canada and mexico .\nwe have included the financial results of kapstone in our corrugated packaging segment since the date of the acquisition .\non september 4 , 2018 , we completed the acquisition ( the 201cschl fcter acquisition 201d ) of schl fcter print pharma packaging ( 201cschl fcter 201d ) .\nschl fcter is a leading provider of differentiated paper and packaging solutions and a german-based supplier of a full range of leaflets and booklets .\nthe schl fcter acquisition allowed us to further enhance our pharmaceutical and automotive platform and expand our geographical footprint in europe to better serve our customers .\nwe have included the financial results of the acquired operations in our consumer packaging segment since the date of the acquisition .\non january 5 , 2018 , we completed the acquisition ( the 201

In [25]:
# Step 2: Initialize FinDER Task
# --------------------------
# In this baseline example, we are using the FinDER task, one of the seven available tasks in this project.
# If you want to use a different task, for example, 'OtherTask', you can change the task initialization as follows:
#
# Example:
# from financerag.tasks import OtherTask
# finder_task = OtherTask()
#
# For this baseline, we proceed with FinDER.
#finder_task = FinDER()
convfinqa_task = ConvFinQA()


hi
convfinqa


In [12]:
finder_task

NameError: name 'finder_task' is not defined

In [3]:
# Step 3: Initialize DenseRetriever model
# -------------------------------------
# Initialize the retrieval model using SentenceTransformers. This model will be responsible
# for encoding both the queries and documents into embeddings.
#
# You can replace 'intfloat/e5-large-v2' with any other model supported by SentenceTransformers.
# For example: 'BAAI/bge-large-en-v1.5', 'Linq-AI-Research/Linq-Embed-Mistral', etc.
encoder_model = SentenceTransformerEncoder(
    model_name_or_path='intfloat/e5-large-v2',
    query_prompt='query: ',
    doc_prompt='passage: ',
)

retrieval_model = DenseRetrieval(
    model=encoder_model
)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/e5-large-v2


In [ ]:
# Step 4: Perform retrieval
# ---------------------
# Use the model to retrieve relevant documents for given queries.

retrieval_result = finder_task.retrieve(
    retriever=retrieval_model
)

# Print a portion of the retrieval results to verify the output.
print(f"Retrieved results for {len(retrieval_result)} queries. Here's an example of the top 5 documents for the first query:")

for q_id, result in retrieval_result.items():
    print(f"\nQuery ID: {q_id}")
    # Sort the result to print the top 5 document ID and its score
    sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

    for i, (doc_id, score) in enumerate(sorted_results[:5]):
        print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

    break  # Only show the first query


INFO:financerag.retrieval.dense:Encoding queries...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

c:\Users\God_YJ\anaconda3\envs\fsi\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...


Batches:   0%|          | 0/217 [00:00<?, ?it/s]

Retrieved results for 216 queries. Here's an example of the top 5 documents for the first query:

Query ID: q00001
  Document 1: Document ID = MSFT20230966, Score = 0.873996376991272
  Document 2: Document ID = MSFT20230216, Score = 0.8645688891410828
  Document 3: Document ID = MSFT20230015, Score = 0.8594436049461365
  Document 4: Document ID = MSFT20230254, Score = 0.8580107092857361
  Document 5: Document ID = MSFT20230155, Score = 0.8534095883369446


In [9]:
# Step 5: Initialize CrossEncoder Reranker
# --------------------------------------
# The CrossEncoder model will be used to rerank the retrieved documents based on relevance.
#
# You can replace 'cross-encoder/ms-marco-MiniLM-L-12-v2' with any other model supported by CrossEncoder.
# For example: 'cross-encoder/ms-marco-TinyBERT-L-2', 'cross-encoder/stsb-roberta-large', etc.
reranker = CrossEncoderReranker(
    model=CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
)


config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

c:\Users\God_YJ\anaconda3\envs\fsi\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\God_YJ\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

c:\Users\God_YJ\anaconda3\envs\fsi\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda


In [10]:
# Step 6: Perform reranking
# -------------------------
# Rerank the top 100 retrieved documents using the CrossEncoder model.
reranking_result = finder_task.rerank(
    reranker=reranker,
    results=retrieval_result,
    top_k=100,  # Rerank the top 100 documents
    batch_size=32
)

# Print a portion of the reranking results to verify the output.
print(f"Reranking results for {len(reranking_result)} queries. Here's an example of the top 5 documents for the first query:")

for q_id, result in reranking_result.items():
    print(f"\nQuery ID: {q_id}")
    # Sort the result to print the top 5 document ID and its score
    sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

    for i, (doc_id, score) in enumerate(sorted_results[:5]):
        print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

    break  # Only show the first query


INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....


Batches:   0%|          | 0/675 [00:00<?, ?it/s]

Reranking results for 216 queries. Here's an example of the top 5 documents for the first query:

Query ID: q00001
  Document 1: Document ID = MSFT20230254, Score = 6.622737884521484
  Document 2: Document ID = MSFT20230233, Score = 6.0889434814453125
  Document 3: Document ID = MSFT20230230, Score = 5.898367404937744
  Document 4: Document ID = MSFT20230236, Score = 5.747088432312012
  Document 5: Document ID = MSFT20230216, Score = 5.488572120666504


In [11]:
# Step 7: Save results
# -------------------
# Save the results to the specified output directory as a CSV file.
output_dir = './results'
finder_task.save_results(output_dir=output_dir)

# Confirm the results have been saved.
print(f"Results have been saved to {output_dir}/FinDER/results.csv")


INFO:financerag.tasks.BaseTask:Output directory set to: ./results\FinDER
INFO:financerag.tasks.BaseTask:Saving top 10 results to CSV file: ./results\FinDER\results.csv
INFO:financerag.tasks.BaseTask:Writing header ['query_id', 'corpus_id'] to CSV.
INFO:financerag.tasks.BaseTask:Top 10 results saved successfully to ./results\FinDER\results.csv


Results have been saved to ./results/FinDER/results.csv
